# Relatório Final - Reembolsos da Câmara dos Deputados

## Introdução

A motivação em desenvolver esse trabalho advém do fato de querer investigar os gastos públicos dos deputados federais brasileiros. Cada político tem o direito a ter reembolso quando gastam seu dinheiro em atividade que permitam "servir melhor o povo". Então é importante que esses gastos sejam monitorados, para saber como cada deputados está utilizando recursos para pedir reembolso com verbas públicas.

Com as diversas notícias e acusações de corrupção dos políticos é de se imaginar que quando envolve dinheiro é provável que se vá encontrar dados suspeitos. 

Sendo assim, com os conhecimentos adquiridos durante o curso, com a utlização da linguagem Python  e com a ajuda das suas bibliotecas, o objetivo do trabalho é analisar os gastos do deputados.

## Dados Utilizados 

## Pré-Processamento

## Perguntas

Foram elaboradas no total sete perguntas para serem respondidas com o conjunto de dados apresentado anteriormente, sendo estas perguntas apresentadas a seguir.

### 1. Qual seria a média dos valores dos recibos para 2021? 

In [ ]:
# ADICIONAR CÓDIGO AQUI

### 2. Existem deputados que são outliers com relação a gastos com gasolina? Se sim, quais? 

In [ ]:
# ADICIONAR CÓDIGO AQUI

### 3. Qual empresa de combustível recebeu mais? Existe alguma relação com possíveis outliers da análise anterior? 

In [ ]:
# ADICIONAR CÓDIGO AQUI

### 4. Existem estabelecimentos com compras de apenas um deputado? 

In [1]:
# ADICIONAR CÓDIGO AQUI

### 5. Qual deputado tem a maior média de gastos? 

In [2]:
# ADICIONAR CÓDIGO AQUI

### 6. Qual foi o deputado que mais gastou em ano eleitoral? 

In [ ]:
# ADICIONAR CÓDIGO AQUI

### 7. Qual é a categoria onde os gastos são maiores? E qual é a categoria onde os gastos são menores?

In [ ]:
# ADICIONAR CÓDIGO AQUI

## Conclusão